In [75]:
import pandas as pd
import numpy as np
import tabula
import geopy
import geopandas
import requests

In [79]:
# read pdf to dataframe - doesn't work for sept22.pdf
dfs = tabula.read_pdf("trafficcount.pdf", output_format='dataframe', pages='7-13', lattice=False)
# df = pd.DataFrame(df)
# df.head()
# tab = pd.DataFrame(tabula.read_pdf('trafficcount.pdf', pages ='7-13')[0])
df = pd.DataFrame()
# for i in dfs:
df = pd.concat(dfs)
df = df.dropna(axis='rows',thresh=3)
df = df[['Count Location', 'Muni', 'FROM', 'TO', 'Year', 'AADT']]
df.head(10)



,Count Location,Muni,FROM,TO,Year,AADT
0,Albany St,C/Ithaca,Traffic Circle,Titus Ave S,2019,"3,239"
1,Albany St,C/Ithaca,Titus Ave S,Titus Ave N,2020,"1,864"
2,Asbury Rd,T/Lansing,Rt 34,Warren Rd,2019,"1,434"
3,Aurora St,C/Ithaca,Lincoln St,Falls St,2019,247
4,Aurora St,C/Ithaca,Marshall St,Lincoln St,2020,"1,371"
5,Aurora St,C/Ithaca,Seneca St,Court St,2020,"4,079"
6,Aurora St,C/Ithaca,Prospect St,State St,2020,"6,746"
7,Aurora St,C/Ithaca,Court St,Marshall St,2020,"1,800"
8,Bald Hill Rd,T/Danby,Comfort Rd,Rt 96B,2020,139
9,Benson Rd,T/Lansing,Rt 34B,Asbury Rd,2021,338


In [77]:
# extract key from env
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
key = os.environ.get("API_KEY")

In [64]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="tompkins-county-map")

# location = geolocator.geocode("Aurora St" + " Ithaca, NY")
# # print(location)
# print((location.latitude, location.longitude))

# # print(location.raw)

(42.433367, -76.494768)


In [113]:
import requests
from urllib.parse import quote

def loc(streetA, streetB):
    addy = streetA + " and " + streetB + ", Ithaca NY"
    addy = quote(addy)
    response = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?address={addy}&key={key}')
    resp_json_payload = response.json()
    return resp_json_payload['results'][0]['geometry']['location']['lat'], resp_json_payload['results'][0]['geometry']['location']['lng']

print(loc(df['Count Location'][0],df['FROM'][0]))

(42.4439614, -76.5018807)


In [103]:
print(resp_json_payload['results'][0]['geometry']['location']['lat'])



42.452093


In [106]:
def func(a,b):
    return a+1,b+1

result = [func(1,2)]
print(type(func(1,2)))
# print(result)

<class 'tuple'>


In [118]:
dh = df.head(4)

result = [[loc(x,y),loc(x,z)] for x, y, z in zip(df['Count Location'],df['FROM'],df['TO'])]

dg = pd.DataFrame(result)
dg.head()

,0,1
0,"(42.4439614, -76.5018807)","(42.4341809, -76.501457)"
1,"(42.4341809, -76.501457)","(42.4341809, -76.501457)"
2,"(42.525045, -76.500204)","(42.5247663, -76.4726724)"
3,"(42.452093, -76.4965809)","(42.45281300000001, -76.496624)"
4,"(42.44618500000001, -76.49598)","(42.452093, -76.4965809)"


In [119]:
dg['avg'] = [((x[0]+y[0])/2,(x[1]+y[1])/2) for x, y in zip(dg[0],dg[1])]
print(dg)
df.head()

                                  0                                1  \
0         (42.4439614, -76.5018807)         (42.4341809, -76.501457)   
1          (42.4341809, -76.501457)         (42.4341809, -76.501457)   
2           (42.525045, -76.500204)        (42.5247663, -76.4726724)   
3          (42.452093, -76.4965809)  (42.45281300000001, -76.496624)   
4    (42.44618500000001, -76.49598)         (42.452093, -76.4965809)   
..                              ...                              ...   
302       (42.5161991, -76.4408409)        (42.5170693, -76.4144638)   
303       (42.5181806, -76.3754441)        (42.5187529, -76.3547083)   
304       (42.4416355, -76.4844907)        (42.5161991, -76.4408409)   
305       (42.4439614, -76.5018807)        (42.5912692, -76.3828528)   
306        (42.904098, -76.5449902)        (42.4439614, -76.5018807)   

                                    avg  
0    (42.439071150000004, -76.50166885)  
1              (42.4341809, -76.501457)  
2        

,level_0,index,Count Location,Muni,FROM,TO,Year,AADT
0,0,0,Albany St,C/Ithaca,Traffic Circle,Titus Ave S,2019,"3,239"
1,1,1,Albany St,C/Ithaca,Titus Ave S,Titus Ave N,2020,"1,864"
2,2,2,Asbury Rd,T/Lansing,Rt 34,Warren Rd,2019,"1,434"
3,3,3,Aurora St,C/Ithaca,Lincoln St,Falls St,2019,247
4,4,4,Aurora St,C/Ithaca,Marshall St,Lincoln St,2020,"1,371"


In [136]:
df = df[['Count Location', 'FROM', 'TO', 'AADT']]
res = list(zip(*dg['avg']))
df['lat'] = res[0]
df['long'] = res[1]
df.head()
df.to_csv('streets.csv')